In [121]:
import datetime
import requests
import json
import pyperclip

def requestGists( username ):
    url = 'https://api.github.com/'

    myResponse = requests.get(url+'users/'+username+'/gists')
    
    # For successful API call, response code will be 200 (OK)
    if(myResponse.ok):

        # Loading the response data into a dict variable
        # json.loads takes in only binary or string variables so using content to fetch binary content
        # Loads (Load String) takes a Json file and converts into python data structure (dict or list, depending on JSON)
        jData = json.loads(myResponse.content.decode('utf-8'))
        return jData
    else:
      # If response code is not ok (200), print the resulting http error code with description
        myResponse.raise_for_status()
        return None

In [88]:
users = {'lacan': 'Olivier Burri', 'romainGuiet': 'Romain Guiet', 'NicoKiaru':'Nicolas Chiaruttini'}

In [124]:
# Get Gistlist from all users
#gists = [ requestGists( user ) for user in users]

# Flatten out
gists = [ gist for user in gists for gist in user]

gists = sorted(gists, key=lambda gist: [ key.lower() for key in gist['files'] ][0], reverse=False)

# Create Table
string = '|Name|Description|Author|Tags|Date|\n|----|-----------|------|----|----|\n'

for gist in gists:
    url = gist['html_url']
    desc = gist['description']
    owner = gist['owner']['login']
    if owner in users:
        owner = users[owner]
    # Remove tags from description
    tags = [tag.strip("#") for tag in desc.split() if tag.startswith("#")]
    
    if "#" in desc:
        desc = desc[:desc.index("#")]
    
    file = [ key for key in gist['files'] ][0] # There is just one file so far
    updated = datetime.datetime.strptime( gist['updated_at'], '%Y-%m-%dT%H:%M:%SZ' )
    
    if "BIOP" in tags:
        string += "| **[[ %s|%s ]]** | %s | %s | %s | %s |\n " % ( url, file, 
                                                   desc, 
                                                   owner, 
                                                   ', '.join( ["`#"+tag+"`" for tag in tags] ), 
                                                   updated.strftime( "%Y/%m/%d" ) )
# Alternative: Create Chapters    
#        string. = ( "== %s ==" % ( file ),
#                   "//Author:// %s" % ( owner ),
#                   "//Description:// %s" % ( desc ),
#                   "//Tags:// %s" % ( ', '.join( ["`#"+tag+"`" for tag in tags] ) ),
#                   "//Last Modified:// %s" % (  ),
#                   "\n"
#                 )
                  
pyperclip.copy( string )

print( "Gist result copied to clipboard" )
        

HTTPError: 403 Client Error: rate limit exceeded for url: https://api.github.com/users/lacan/gists